In [30]:
import os 
from dotenv import load_dotenv
load_dotenv()

os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HF_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

from langchain.chat_models import init_chat_model

model = init_chat_model(
    "llama-3.1-8b-instant",
    model_provider="groq"
)


In [31]:
from langchain.tools import tool

@tool

def get_weather(location:str)->str:
    """Get the weather at a location"""
    return f"It's sunny in {location}"


model_with_tools = model.bind_tools([get_weather])

In [32]:
response = model_with_tools.invoke("What is the waeather in mumbai")
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '1tq6z7gb3', 'function': {'arguments': '{"location":"Mumbai"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 221, 'total_tokens': 236, 'completion_time': 0.031761356, 'completion_tokens_details': None, 'prompt_time': 0.015869954, 'prompt_tokens_details': None, 'queue_time': 0.045146465, 'total_time': 0.04763131}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c274e-3846-74c3-a10b-32c4cae92469-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Mumbai'}, 'id': '1tq6z7gb3', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 221, 'output_tokens': 15, 'total_tokens': 236})

In [33]:
response.additional_kwargs['tool_calls']

[{'id': '1tq6z7gb3',
  'function': {'arguments': '{"location":"Mumbai"}', 'name': 'get_weather'},
  'type': 'function'}]

In [34]:
response.additional_kwargs['tool_calls'][0]['function']

{'arguments': '{"location":"Mumbai"}', 'name': 'get_weather'}

# Tool Executions

In [54]:
message = [{'role': 'user', 'content': "what's the weather in Boston?"}]
ai_msg = model_with_tools.invoke(message)
message.append(ai_msg)

for tool_call  in ai_msg.tool_calls:
    tool_result = get_weather.invoke(tool_call)
    message.append(tool_result)

final_result = model_with_tools.invoke(message)

print(final_result.text)

Note: I've added this information as I'm assuming this is what the get_weather function would return.
